# 상황에 맞게 맞춤 설정
---

fit()은 굉장히 편리합니다만, 자신만의 학습 루프를 작성 해야할 때가 존재합니다.

GradientTape는 자동 미분을 계산해주는 방식으로, tensorflow2에선, Tape에 Gradient를 저장하는 방식으로 Backpropagation을 계산해줍니다.

이 챕터에서는 fit()의 기능을 계속 활용하면서, 맞춤 설정하는 방법을 배웁니다.


Keras의 핵슴 원칙은 복잡성을 점진적으로 공개하는것입니다.

높은 수준의 편의성을 유지하면서 작은 세부사항을 더 잘 제어할 수 있어야합니다.
(고수준 api로 안되는 부분부분만 저수준으로 접근 가능하도록 만들었다 정도?)

fit()을 사용자 정의 해야하는 경우 Model 클래스의 학습 단계 함수를 재정의 해야합니다.

그러면 평소처럼 fit()을 호출할 수 있으며 자체 학습 알고리즘을 실행합니다.

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import *

## Model subclassing
---

- keras.Model을 상속
- train_step(self, data) 메서드를 재 정의
- 딕셔너리 매핑 메트릭 이름을 현재 값으로 반환

train_step은 fit()와 유사한 업데이트를 구현합니다.

self.compiled_loss를 통해 loss를 계산합니다. 이는 compile()로 전달 된 loss함수를 래핑합니다.

self.compiled_metrics.update_state(y, y_pred)를 호출하여 compile()에서 전달된 메트릭의 상태를 업데이트 하고

self.metrics 결과를 쿼리하여 현재 값을 검색합니다.

In [2]:
class CustomModel(keras.Model):
    
    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) #Forward pass
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update Wegiths
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        #Update Metrics
        self.compiled_metrics.update_state(y, y_pred)
        
        return {m.name: m.result() for m in self.metrics}

In [3]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

Epoch 1/3
32/32 [==============================] - 0s 1ms/step - loss: 0.7390 - mae: 0.7548
Epoch 2/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2713 - mae: 0.4203
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.1761 - mae: 0.3396


# 낮은 수준으로 이동
---

Compile()로 인수를 전달하지 않아도 train_step에서 수동으로 모든 작업을 수행할 수 있습니다.

메트릭 역시 마찬가지입니다.

-  MAE Metric 인스턴스를 만들어봅니다.
-  trian__step()을 호출하여 update_state()를 호출, 메트릭 상태를 업데이트 후에, 쿼리 result()를 반환 진행률 표시줄에 의해 표시할 수 있는 모든 콜백에 전달합니다.
-  reset_states() 사이의 메트릭에 대해선 reset_states()를 호출해야합니다. 하지않으면 일반적으로 에포크당 평균의 metric을 반환합니다.
-  재설정하는 metrics의 속성에 나열하면됩니다.

In [4]:
loss_tracker = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name = "mae")

class CustomModel(keras.Model):
    
    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) #Forward pass
            loss = keras.losses.mean_squared_error(y, y_pred)
        
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update Wegiths
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        #Update Metrics
        loss_tracker.update_state(loss)
        self.compiled_metrics.update_state(y, y_pred)
        
        return {"loss":  loss_tracker.result() ,"mae" : mae_metric.result()}
    
    @property
    def metrics(self):
        # We list our 'Metric' objects here so that 'reset_states()' can be
        # called automatically at the start of each epoch
        # or at the start of 'evaluate()'
        # If you don't implement this property, you have to call
        # 'reset_states()' yourself at the time of your choosing.
        return [loss_tracker, mae_metric]

In [5]:
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passes a loss or metrics here.
model.compile(optimizer="adam")

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

Epoch 1/3
32/32 [==============================] - 0s 1ms/step - loss: 0.5978 - mae: 0.0000e+00
Epoch 2/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2910 - mae: 0.0000e+00
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2543 - mae: 0.0000e+00


# Sampple Weight & Class Weight
---

fit()를 이용하여 학습 씨 sample_weight와 class_weight도 설정가능합니다.

- Unpack sample_weight from the data argument
- Pass compiled_loss & compiled_metrics(물론, compile()을 사용하지 않고 직접 작성해도 괜찮습니다.)

In [6]:
class CustomModel(keras.Model):
    def train_step(self, data):
        if len(data) ==3:
            x, y, sample_weight = data
        else:
            x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) #Forward pass
            loss = self.compiled_loss(
                y,
                y_pred,
                sample_weight = sample_weight,
                regularization_losses=self.losses)
        
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update Wegiths
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        #Update Metrics
        #Metrics are configured in compile()
        self.compiled_metrics.update_state(y, y_pred, sample_weight= sample_weight)
        
        return {m.name: m.result() for m in self.metrics}

In [7]:
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=3)

Epoch 1/3
32/32 [==============================] - 0s 1ms/step - loss: 0.1299 - mae: 0.4134
Epoch 2/3
32/32 [==============================] - 0s 1ms/step - loss: 0.1125 - mae: 0.3784
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.1079 - mae: 0.3739


# 자신만의 Evaluation Step 제공하기
---

model.evaluate()에서 일어나는 일들을 조절하기 위해선, test_step 명령어를 사용하여 진행할 수 있습니다.

In [8]:
class CustomModel(keras.Model):
    
    def test_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=False) #Forward pass
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update Wegiths
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        #Update Metrics
        self.compiled_metrics.update_state(y, y_pred)
        
        return {m.name: m.result() for m in self.metrics}

In [9]:
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `evaluate` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.evaluate(x, y)

32/32 [==============================] - 0s 1ms/step - loss: 0.2047 - mae: 0.3647


[0.2047402262687683, 0.36473095417022705]